# Coreset Selection for Unlearning

Coresets reduce the amount of forget data we need to process while still achieving strong unlearning. This notebook compares **random** vs **gradient-norm** selection.

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

from erasus.unlearners import ErasusUnlearner
from erasus.metrics.metric_suite import MetricSuite
import erasus.strategies
import erasus.selectors

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model = nn.Sequential(nn.Linear(64, 128), nn.ReLU(), nn.Linear(128, 10)).to(device)
forget_loader = DataLoader(
    TensorDataset(torch.randn(100, 64), torch.randint(0, 10, (100,))),
    batch_size=16, shuffle=True
)
retain_loader = DataLoader(
    TensorDataset(torch.randn(400, 64), torch.randint(0, 10, (400,))),
    batch_size=16, shuffle=True
)

for selector_name in ["random", "gradient_norm"]:
    u = ErasusUnlearner(
        model=nn.Sequential(nn.Linear(64, 128), nn.ReLU(), nn.Linear(128, 10)).to(device),
        strategy="gradient_ascent",
        selector=selector_name,
        device=device,
        selector_kwargs={"prune_ratio": 0.3},
        strategy_kwargs={"lr": 1e-3},
    )
    u.fit(forget_data=forget_loader, retain_data=retain_loader, epochs=2)
    m = MetricSuite(["accuracy"]).run(u.model, forget_loader, retain_loader)
    acc = m.get("accuracy", "N/A")
    acc_str = f"{acc:.4f}" if isinstance(acc, (int, float)) else str(acc)
    print(f"Selector {selector_name}: accuracy = {acc_str}")